<a href="https://colab.research.google.com/github/klfontus/Coursera_Capstone/blob/main/Week_3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Defining Foursquare Credenials and Version

# <center> Segmenting and Clustering Toronto Neighborhoods

## Creating a DataFrame of Toronto Neighborhoods and their Postal Codes






##### Import necessary libraries

In [2]:
#!pip install bs4
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


##### URL for Toronto postal codes

In [3]:
#Wikipedia page with table containing postal codes of Toronto neighborhoods
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

##### Scraping data from Wikipedia page

In [4]:
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')
soup.prettify();

##### Locating the table and isolating its rows

In [5]:
table = soup.find('table')
cols = ['PostalCode', 'Borough', 'Neighborhood']
#print(table)
table_rows = table.find_all('tr')
#table_rows[0]

##### Initializing the data frame and adding table data

In [6]:
neighborhood_data = pd.DataFrame(columns=cols)

#cell = table_rows.find_all('td')
for row in table_rows:
  for col in row.find_all('td'):
    
    #print(col)
    if col.span.text=='Not assigned':
        continue
    #print(col.span.string)
    postalCode = col.p.text[:3]
    borough = (col.span.text).split('(')[0]
    neighborhood = (((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    neighborhood_data.loc[len(neighborhood_data)] = [postalCode, borough, neighborhood]
neighborhood_data['Borough']=neighborhood_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#    if col.find('i'): continue
#    postalCode = col.find('b').string
#    try:
#      names = col.find_all('a')
      #print(names)
#      borough = names[0].string
      #print(borough)
#      hoods=""
#      for i in range(1,len(names)):
#        hoods = hoods + names[i].string + ", "
#      neighborhood = hoods[:len(hoods) - 2]
      #print(postalCode, " ", borough, " ", neighborhood)
#      neighborhood_data.loc[len(neighborhood_data)] = [postalCode, borough, neighborhood]
#    except:
#      continue

neighborhood_data["PostalCode"] = neighborhood_data['PostalCode'].astype('string')
neighborhood_data["Borough"] = neighborhood_data['Borough'].astype('string')
neighborhood_data["Neighborhood"] = neighborhood_data['Neighborhood'].astype('string')

neighborhood_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
neighborhood_data.shape

(103, 3)

## Getting Geo Data for Toronto Boroughs

##### Getting Postal Code Geo Data

In [8]:
#GEOCODER WOULD NOT WORK
#!pip install geocoder
#import geocoder 

In [9]:
!wget -q -O 'postal_geo.csv' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

postal_geo = pd.read_csv('postal_geo.csv')
postal_geo["Postal Code"] = postal_geo['Postal Code'].astype('string')
postal_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Extracting Latitude and Longitude from Geo Data and Adding it to Neighborhood Data Frame





In [10]:
df_lat = []
df_long = []

# loop until you get the coordinates
for i in neighborhood_data["PostalCode"]:
  #print(postal_geo[postal_geo["Postal Code"] == i])
  code = postal_geo.loc[postal_geo["Postal Code"] == i]#.values
  #code.shape()
  #print(code.iloc[0]["Latitude"])
  df_lat.append(code.iloc[0]["Latitude"])
  df_long.append(code.iloc[0]["Longitude"])

In [11]:
neighborhood_data["Latitude"] = df_lat
neighborhood_data["Longitude"] = df_long
neighborhood_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [12]:
neighborhood_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    string 
 1   Borough       103 non-null    string 
 2   Neighborhood  103 non-null    string 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), string(3)
memory usage: 4.8 KB


# Exploring Toronto Neighborhoods

##### Creating a map of Toronto with the Neighborhoods Marked

In [13]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data["Latitude"], 
                                                  neighborhood_data["Longitude"], 
                                                  neighborhood_data["Borough"], 
                                                  neighborhood_data["Neighborhood"]):
    label = '{}: {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Making a List of Boroughs that Contain "Toronto" and Making a DataFrame of Neighborhoods in those Boroughs

In [15]:
tor_boroughs = []
for i in set(neighborhood_data["Borough"]):
  if "Toronto" in i:
    tor_boroughs.append(i)
#tor_boroughs

In [16]:
tor_neigh_data = pd.DataFrame(columns=neighborhood_data.columns)
for index, row in neighborhood_data.iterrows():
  if row["Borough"] in tor_boroughs:
    #print(row)
    tor_neigh_data.loc[len(tor_neigh_data)] = row

#tor_neigh = neighborhood_data.loc["Toronto" in neighborhood_data['Borough']]
tor_neigh_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


##### Map of "Toronto" Neighborhoods

In [17]:
map_toronto_neigh = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(tor_neigh_data["Latitude"], 
                                                  tor_neigh_data["Longitude"], 
                                                  tor_neigh_data["Borough"], 
                                                  tor_neigh_data["Neighborhood"]):
    label = '{}: {}'.format(borough, neighborhood)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neigh)  
    
map_toronto_neigh

##### Initializing Foursquare Info

In [18]:
CLIENT_ID = 'VLVGEEHYISQQVPNBN4MXVXRYYFF1F424SYHP1P51VBNZ1CDI' # your Foursquare ID
CLIENT_SECRET = 'FU1ZPCQCQNTWLF5LJH4IGECWHUT1DZIFEKWWXX4Y1WQKUDN4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

##### Looking at the Toronto Bouroughs that We will be Comparing 

In [19]:
print(tor_boroughs)

['East Toronto', 'Downtown Toronto', 'Downtown Toronto Stn A', 'Central Toronto', 'East York/East Toronto', 'East Toronto Business', 'West Toronto']


###### We find the Coordinates of Each Borough by Taking the Average of the Neighborhood Coordinates 

In [20]:
#tor_neigh_geo = pd.DataFrame(columns=["Borough", "Latitude", "Longitude"])
tor_bor_geo = tor_neigh_data.groupby(["Borough"]).mean().reset_index()
tor_bor_geo.head()

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654624,-79.384184
2,Downtown Toronto Stn A,43.646435,-79.374846
3,East Toronto,43.671110,-79.325428
4,East Toronto Business,43.662744,-79.321558


In [21]:
for lat, lng, label in zip(tor_bor_geo['Latitude'], tor_bor_geo['Longitude'], tor_bor_geo['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neigh)  
    
map_toronto_neigh

##### First let's look at the Venue Data for Downtown Toronto

In [22]:
dtToronto = "Downtown Toronto"

borough_latitude = tor_bor_geo.loc[1, 'Latitude'] # neighborhood latitude value
borough_longitude = tor_bor_geo.loc[1, 'Longitude'] # neighborhood longitude value

borough_name = dtToronto

In [23]:
# type your answer here
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e000aaacbd503c805a7721'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 179,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,The Elm Tree Restaurant,Modern European Restaurant,43.657397,-79.383761
3,KAKA,Japanese Restaurant,43.657457,-79.384192
4,Japango,Sushi Restaurant,43.655268,-79.385165


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.groupby(['categories']).count()

67 venues were returned by Foursquare.


,name,lat,lng
categories,,,
Art Gallery,1,1,1
Art Museum,1,1,1
Bank,1,1,1
Bookstore,1,1,1
Bubble Tea Shop,2,2,2
Burger Joint,1,1,1
Café,1,1,1
Clothing Store,6,6,6
Cocktail Bar,1,1,1


###### Looking at the different types of Venues in Downtown Toronto, the borough seems to be mostly a shopping district. There are 6 clothing stores, and many different eateries for shoppers to choose from. 

##### Now let's compare all of the boroughs to each other by running the same function on all of them

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(tor_bor_geo["Borough"], tor_bor_geo["Latitude"], tor_bor_geo["Longitude"])

Central Toronto
Downtown Toronto
Downtown Toronto Stn A
East Toronto
East Toronto Business
East York/East Toronto
West Toronto


###### Here is the size of the data frame and the amount of venues in each Borough

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(277, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.70198,-79.398954,Balsamico,43.701505,-79.397162,Italian Restaurant
1,Central Toronto,43.70198,-79.398954,La Salumeria,43.701255,-79.397117,Deli / Bodega
2,Central Toronto,43.70198,-79.398954,Five Doors North,43.702236,-79.397526,Italian Restaurant
3,Central Toronto,43.70198,-79.398954,Tabülè,43.700932,-79.397078,Middle Eastern Restaurant
4,Central Toronto,43.70198,-79.398954,Noorden Food x Bar,43.702814,-79.397645,Restaurant


In [29]:
toronto_venues.groupby('Borough').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 117 uniques categories.


###### Next we do one-hot encoding on the boroughs and their venues to analyze the data

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Cheese Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Egyptian Restaurant,...,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Opera House,Optical Shop,Pakistani Restaurant,Park,Pharmacy,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Yoga Studio
0,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Cheese Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Egyptian Restaurant,...,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Opera House,Optical Shop,Pakistani Restaurant,Park,Pharmacy,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sushi Restaurant,Syrian Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.000000,0.000000,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.029851,0.014925,0.000000,0.014925,0.000000,0.000000,0.089552,0.014925,0.104478,0.000000,0.014925,0.014925,0.000000,0.029851,0.000000,0.000000,0.000000,0.014925,0.014925,0.029851,0.000000,0.000000,...,0.014925,0.000000,0.014925,0.014925,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.029851,0.000000,0.014925,0.014925,0.000000,0.014925,0.000000,0.029851,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.000000,0.014925,0.014925,0.000000,0.029851,0.000000,0.014925,0.014925,0.014925,0.000000
2,Downtown Toronto Stn A,0.010101,0.010101,0.020202,0.000000,0.000000,0.000000,0.010101,0.030303,0.000000,0.000000,0.010101,0.030303,0.010101,0.010101,0.010101,0.020202,0.000000,0.000000,0.000000,0.000000,0.030303,0.020202,0.010101,0.010101,0.030303,0.121212,0.010101,0.000000,0.010101,0.000000,0.010101,0.020202,0.000000,0.010101,0.000000,0.000000,0.010101,0.010101,0.000000,...,0.010101,0.010101,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.020202,0.010101,0.000000,0.000000,0.010101,0.030303,0.000000,0.000000,0.030303,0.010101,0.040404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000

###### Now let's list the top 5 venue types for each Borough and put them into Data Frames

In [33]:
num_top_venues = 5

for borough in toronto_grouped['Borough']:
    print("----"+borough+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
                   venue  freq
0     Italian Restaurant  0.19
1       Sushi Restaurant  0.11
2             Restaurant  0.07
3            Coffee Shop  0.07
4  Indonesian Restaurant  0.04


----Downtown Toronto----
              venue  freq
0       Coffee Shop  0.10
1    Clothing Store  0.09
2    Cosmetics Shop  0.03
3  Sushi Restaurant  0.03
4             Plaza  0.03


----Downtown Toronto Stn A----
                venue  freq
0         Coffee Shop  0.12
1  Italian Restaurant  0.04
2  Seafood Restaurant  0.04
3                 Pub  0.03
4               Hotel  0.03


----East Toronto----
                  venue  freq
0     Indian Restaurant  0.22
1         Grocery Store  0.11
2                  Café  0.07
3    Italian Restaurant  0.04
4  Pakistani Restaurant  0.04


----East Toronto Business----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2                 Spa  0.06
3         Pizza Place  0.06
4      Farmers Market  0.06


-

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###### We see that areas such as Downtown and Central Toronto have more shopping and eating spots, while areas like East Toronto and East York are more suburban and have places such as grocery stores and pharmacies

In [36]:
def return_most_common_venues(row, num_top_venues):
    """Sorts the functions in descending order"""
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### Now we will put the names of the top 10 venue types into  DataFrame

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Indonesian Restaurant,Convenience Store,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant
1,Downtown Toronto,Coffee Shop,Clothing Store,Hotel,Seafood Restaurant,Bubble Tea Shop,Plaza,Theater,Cosmetics Shop,Diner,Middle Eastern Restaurant
2,Downtown Toronto Stn A,Coffee Shop,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Pub,Restaurant,Bakery,Japanese Restaurant,Beer Bar,Hotel
3,East Toronto,Indian Restaurant,Grocery Store,Café,Pakistani Restaurant,Park,Pizza Place,Egyptian Restaurant,Brewery,Convenience Store,Shopping Plaza
4,East Toronto Business,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Recording Studio


In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Indonesian Restaurant,Convenience Store,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant
1,Downtown Toronto,Coffee Shop,Clothing Store,Hotel,Seafood Restaurant,Bubble Tea Shop,Plaza,Theater,Cosmetics Shop,Diner,Middle Eastern Restaurant
2,Downtown Toronto Stn A,Coffee Shop,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Pub,Restaurant,Bakery,Japanese Restaurant,Beer Bar,Hotel
3,East Toronto,Indian Restaurant,Grocery Store,Café,Pakistani Restaurant,Park,Pizza Place,Egyptian Restaurant,Brewery,Convenience Store,Shopping Plaza
4,East Toronto Business,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Recording Studio


###### Now we will run K-Means on the boroughs to put them into clusters. We will do three clusters since we only have seven boroughs

In [51]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 1, 2], dtype=int32)

In [76]:
#borough_venues_sorted.drop('Cluster Labels', 1, inplace=True)
#borough_venues_sorted.head()

In [64]:
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_bor_geo

toronto_merged = toronto_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head()

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.701980,-79.398954,2,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Indonesian Restaurant,Convenience Store,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant
1,Downtown Toronto,43.654624,-79.384184,2,Coffee Shop,Clothing Store,Hotel,Seafood Restaurant,Bubble Tea Shop,Plaza,Theater,Cosmetics Shop,Diner,Middle Eastern Restaurant
2,Downtown Toronto Stn A,43.646435,-79.374846,2,Coffee Shop,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Pub,Restaurant,Bakery,Japanese Restaurant,Beer Bar,Hotel
3,East Toronto,43.671110,-79.325428,0,Indian Restaurant,Grocery Store,Café,Pakistani Restaurant,Park,Pizza Place,Egyptian Restaurant,Brewery,Convenience Store,Shopping Plaza
4,East Toronto Business,43.662744,-79.321558,2,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Recording Studio


In [65]:
toronto_merged.shape

(7, 14)

###### Now we map the boroughs to show the differenct clusters

In [70]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Cluster 1 Analysis

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,Indian Restaurant,Grocery Store,Café,Pakistani Restaurant,Park,Pizza Place,Egyptian Restaurant,Brewery,Convenience Store,Shopping Plaza


##### Looking at this cluster, we see that the most common restaurants are Indian, Pakistani, and Egyptian cusines. Grocery stores and parks are also in the top 5 venues. This leads me to believe that in this borough, there might be a larger immigrant population full of families that would use stores and parks. The presence of the goricery stores, parks, convenience stores, and shopping plaza makes me believe that this is a more suburban area.

##### Cluster 2 Analysis

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,East York/East Toronto,Park,Convenience Store,Fish Market,Cosmetics Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner


###### Convenience stores and parks are at the top of the list here, suggesting there are a lot of families, possibly with kids, who live in this area. Plus the cosmetic shops and department store mean that this is also a suburban area or small shopping district. Having both Cuban restaurants and deli/bodega in the top ten suggests the presence of a Hispanic or Latino population. Looking at the map, clusters 1 and 2 are close together, which supports my idea that there is one large suburban area of central Toronto.

##### Cluster 3

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Indonesian Restaurant,Convenience Store,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant
1,Downtown Toronto,Coffee Shop,Clothing Store,Hotel,Seafood Restaurant,Bubble Tea Shop,Plaza,Theater,Cosmetics Shop,Diner,Middle Eastern Restaurant
2,Downtown Toronto Stn A,Coffee Shop,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Pub,Restaurant,Bakery,Japanese Restaurant,Beer Bar,Hotel
4,East Toronto Business,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Recording Studio
6,West Toronto,Restaurant,Gift Shop,Pub,Breakfast Spot,Bookstore,Café,Brewery,Gourmet Shop,Coffee Shop,Italian Restaurant


###### This is the largest cluster. At first look, they do look very similar. A variety of restaurants and small food shops make up the majority of the lists. There are pubs and cocktail bars, meaning that there is nightlife. There are also  shopping areas with gift shops, bookstores, etc and cafes for casual dining. This points to this cluster being a tourist/major shopping area. There are also no parks, only gardens in East Toronto Business who's ,ost common venue is actually a light rail station, leading me to believe that this is supposed to represent an urban cluster. Looking at the map, you might think that West and Central Toronto would be too far from the other nodes to be in their cluster, but looking at their common venues, they fit in with the shopping and restaurant theme.

#### My analysis leads me to believe that the main Toronto boroughs are made up of an enourmous urban area, with smaller suburbs where families can live more comfortably than they could in the middle of the large city. If I were moving to Toronto, I think I would rather live in Clusters 1 or 2, and then reserve Cluster 3 just for shopping and for entertaining friends who visit me.